<!--I'm writing this post as part of a blog. When I ask you to rephrase things bare this in mind. Also make sure you return requested rephrasing in markdown code so that I can easily copy and paste 
-->

# GPTRec Implementation

Following up from my previous post on the [GPTRec paper](https://andrewboney.github.io/andrew_boney_blog/posts/GPTRec%20Paper%20Read/), I want to implement a GPTRec-style system.

The original author has existing implementations across a couple of repos: [gptrec_rl](https://github.com/asash/gptrec_rl) and [bert4rec_repro](https://github.com/asash/bert4rec_repro). However, these are broader in scope than I need, and are implemented in TensorFlow (boo!!!).

Instead, I'll be implementing this using a framework I'm building: [rec](https://github.com/AndrewBoney/rec). This is a work in progress aiming to provide an end-to-end implementation of the kinds of recommendation systems used in industry—encompassing the whole lifecycle from data prep to deployment. While there's still plenty to add, the baseline should now be robust enough to work at scale. I'll likely use this framework in future posts too.

In this implementation, I'm going to focus on the GPTRec architecture - ignoring the sub-item tokenisation and Next-K prediction aspects of the paper.

## Get repo

In [ ]:
!git clone https://github.com/AndrewBoney/rec.git && cd rec && git checkout 7a6475f


fatal: destination path 'rec' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.insert(0, 'rec')

# Data

First, we'll use the [rec](https://github.com/AndrewBoney/rec) framework to prepare the data. 

## Generate data

I'll use the [MovieLens 1M dataset](https://grouplens.org/datasets/movielens/1m/)—a classic benchmark in recommendation systems research containing 1 million ratings from 6,000 users on 4,000 movies. The `rec` framework includes a [data preparation module](https://github.com/AndrewBoney/rec/blob/main/rec/data_prep/movielens.py) that downloads and processes this dataset into a consistent format for training.

In [ ]:
import pandas as pd

import os

from rec.data_prep.movielens import generate_movielens_1m
from rec.common.data import DataPaths

root_folder = "../../assets/movielens_rec_data"

generate_movielens_1m(output_dir = root_folder)

In [ ]:
paths = DataPaths(
    users_path = os.path.join(root_folder, "prepared", "users.parquet"),
    items_path = os.path.join(root_folder, "prepared", "items.parquet"),
    interactions_train_path = os.path.join(root_folder, "prepared", "interactions_train.parquet"),
    interactions_val_path = os.path.join(root_folder, "prepared", "interactions_val.parquet")
)

In [ ]:
users = pd.read_parquet(paths.users_path)
print(users.head())

  user_id   age gender age_group occupation    zip zip_prefix
0       1   1.0      F   age_1.0     occ_10  48067        480
1       2  56.0      M  age_56.0     occ_16  70072        700
2       3  25.0      M  age_25.0     occ_15  55117        551
3       4  45.0      M  age_45.0      occ_7  02460        024
4       5  25.0      M  age_25.0     occ_20  55455        554


In [ ]:
items = pd.read_parquet(paths.items_path)
print(items.head())

  item_id                        genres                 genre_grouped  \
0       1   Animation|Children's|Comedy                         other   
1       2  Adventure|Children's|Fantasy  Adventure|Children's|Fantasy   
2       3                Comedy|Romance                Comedy|Romance   
3       4                  Comedy|Drama                  Comedy|Drama   
4       5                        Comedy                        Comedy   

                                title                    title_raw  year  \
0                    Toy Story (1995)                    Toy Story  1995   
1                      Jumanji (1995)                      Jumanji  1995   
2             Grumpier Old Men (1995)             Grumpier Old Men  1995   
3            Waiting to Exhale (1995)            Waiting to Exhale  1995   
4  Father of the Bride Part II (1995)  Father of the Bride Part II  1995   

  year_bucket  
0  year_1990s  
1  year_1990s  
2  year_1990s  
3  year_1990s  
4  year_1990s  


In [ ]:
interactions_train = pd.read_parquet(paths.interactions_train_path)
print(interactions_train.head())

  user_id item_id  rating   timestamp
0       1    1193       5  2000-12-31
1       1     661       3  2000-12-31
2       1     914       3  2000-12-31
3       1    3408       4  2000-12-31
4       1    2355       5  2001-01-06


In [ ]:
interactions_val = pd.read_parquet(paths.interactions_val_path)
print(interactions_val.head())

  user_id item_id  rating   timestamp
0      36    1266       5  2002-12-22
1      36    2713       1  2002-12-22
2      36     595       4  2002-12-22
3      36     247       4  2002-12-22
4      36    1295       4  2002-12-22


## Define Feature Config

This defines the cols used in the dataset, and the types of features they should be converted into.

In [ ]:
from rec.common.data import FeatureConfig

feature_config = FeatureConfig(
    user_id_col = "user_id", 
    item_id_col = "item_id",
    user_cat_cols = [],
    item_cat_cols = [], 
    interaction_user_col = "user_id",
    interaction_item_col = "item_id", 
    interaction_time_col = "timestamp"
)

## Build Encoders

Before training, we need to convert raw IDs (like `u_000001` or `i_000042`) into integer indices that can be used for embedding lookups—much like tokenization in NLP. The `build_encoders` function creates a `CategoryEncoder` for each categorical column, mapping each unique value to an integer index while reserving 0 for unknown values.

In [ ]:
from rec.common.data import build_encoders

user_encoders, item_encoders = build_encoders(
    users_path = paths.users_path,
    items_path = paths.items_path,
    interactions_path = paths.interactions_train_path,
    feature_cfg = feature_config
)

In [ ]:
user_encoders, item_encoders

({'user_id': <rec.common.data.CategoryEncoder at 0x76e62cbc0f80>},
 {'item_id': <rec.common.data.CategoryEncoder at 0x76e62cbc11c0>})

## Build Cardinalities

We also need to define the feature cardinalities, i.e. the number of unique values for each categorical feature. This is used to determine the size of the embeddings. 

In [ ]:
from rec.common.train import build_cardinalities

user_cardinalities = build_cardinalities(user_encoders, [feature_config.user_id_col])
item_cardinalities = build_cardinalities(item_encoders, [feature_config.item_id_col])

In [ ]:
user_cardinalities, item_cardinalities

({'user_id': 6041}, {'item_id': 3884})

In this case we only need `user_id` and `item_id`. Note that the cardinality is the number of unique values, plus one for the unknown value.

## Build User / Item Map

Sequential recommendation models like GPTRec learn from the *order* in which users interact with items—predicting the next item based on the sequence of previous ones. To train such models, we need to transform our flat interaction table into an ordered mapping: for each user, a chronologically sorted list of item IDs.

While the `rec` framework includes a [`build_user_item_map`](https://github.com/AndrewBoney/rec/blob/7a6475f946ebb6f9285572ef6b2d53dc5a05c38f/rec/common/train.py#L118) function, this was designed for non-sequential models where interaction order doesn't matter—it simply collects the set of items each user has interacted with. For GPTRec, we need a modified version that preserves temporal ordering by sorting on the timestamp column. I'll likely integrate this into the framework in a future update.

In [ ]:
from typing import Dict, List

from rec.common.io import read_parquet_batches
from rec.common.data import FeatureConfig, CategoryEncoder

def build_user_item_map_ordered(
    interactions_path: str,
    feature_cfg: FeatureConfig,
    user_encoders: Dict[str, CategoryEncoder],
    item_encoders: Dict[str, CategoryEncoder],
    chunksize: int = 200_000,
) -> Dict[int, List[int]]:
    """Build user->items map ordered by timestamp (ascending)."""
    user_to_items: Dict[int, List[tuple]] = {}  # uid -> [(timestamp, item_id), ...]
    
    for chunk in read_parquet_batches(interactions_path, chunksize):
        user_ids = user_encoders[feature_cfg.user_id_col].transform(
            chunk[feature_cfg.interaction_user_col].astype(str).tolist()
        )
        item_ids = item_encoders[feature_cfg.item_id_col].transform(
            chunk[feature_cfg.interaction_item_col].astype(str).tolist()
        )
        timestamps = chunk[feature_cfg.interaction_time_col].tolist()
        
        for uid, iid, ts in zip(user_ids, item_ids, timestamps):
            uid, iid = int(uid), int(iid)
            if uid not in user_to_items:
                user_to_items[uid] = []
            user_to_items[uid].append((ts, iid))
    
    # Sort by timestamp and extract just the item ids
    return {
        uid: [iid for _, iid in sorted(items)]
        for uid, items in user_to_items.items()
    }

train_user_item_map = build_user_item_map_ordered(
    paths.interactions_train_path,
    feature_config,
    user_encoders,
    item_encoders,
)

val_user_item_map = build_user_item_map_ordered(
    paths.interactions_val_path,
    feature_config,
    user_encoders,
    item_encoders,
)

In [ ]:
print(train_user_item_map[36])

[1, 11, 21, 30, 32, 34, 47, 143, 171, 177, 195, 221, 230, 231, 244, 254, 314, 326, 353, 377, 438, 446, 477, 497, 521, 548, 584, 585, 586, 594, 643, 700, 771, 776, 842, 1064, 1066, 1082, 1112, 1120, 1121, 1157, 1173, 1225, 1230, 1231, 1240, 1251, 1255, 1259, 1266, 1278, 1301, 1336, 1352, 1373, 1375, 1376, 1383, 1425, 1449, 1456, 1483, 1492, 1506, 1530, 1534, 1540, 1544, 1596, 1608, 1618, 1631, 1696, 1761, 1764, 1807, 1808, 1814, 1839, 1841, 1855, 1895, 1900, 1932, 1944, 1955, 1997, 2010, 2013, 2026, 2032, 2040, 2069, 2077, 2087, 2175, 2177, 2180, 2182, 2226, 2244, 2253, 2284, 2287, 2303, 2327, 2338, 2339, 2350, 2356, 2364, 2365, 2401, 2422, 2434, 2473, 2503, 2512, 2531, 2537, 2560, 2603, 2615, 2619, 2625, 2631, 2632, 2634, 2638, 2648, 2654, 2678, 2693, 2694, 2695, 2723, 2729, 2790, 2847, 2850, 2891, 2905, 2919, 2967, 2969, 2971, 2984, 2992, 3004, 3013, 3019, 3046, 3088, 3107, 3108, 3145, 3185, 3187, 3195, 3230, 3233, 3240, 3290, 3293, 3356, 3436, 3437, 3457, 3458, 3484, 3510, 3523, 3555

In [ ]:
print(val_user_item_map[36])

[245, 294, 592, 1247, 1276, 1654, 2201, 2301, 2626, 2645, 3106, 3571, 3718]


In a future iteration, I may extend this to include timestamps in the output mapping. This would enable time-based positional embeddings—encoding *when* interactions occurred rather than just their relative order. For now, I'll keep things simple with index-based positional embeddings.

## Build Feature Store

The [`FeatureStore`](https://github.com/AndrewBoney/rec/blob/main/rec/common/data.py#L174) class provides efficient feature lookup for users and items during training and inference. Rather than repeatedly encoding features on-the-fly, it pre-encodes all user and item features into tensors at initialization—storing them in memory for fast indexed access.

Key functionality:
- **Pre-encoded tensors**: All categorical and dense features are encoded once and stored as PyTorch tensors with zero-padding at index 0 (for unknown/missing values)
- **Index mappings**: Maintains `user_index` and `item_index` dictionaries that map encoded IDs to their row positions in the feature tensors
- **Batch lookups**: `get_user_features()` and `get_item_features()` retrieve all features for a batch of IDs in a single operation
- **Item catalog access**: `get_all_item_features()` and `get_all_item_ids()` provide full item catalog access—useful for scoring all items during inference

In [ ]:
from rec.common.data import FeatureStore

fs = FeatureStore(
    user_df=users,
    item_df=items,
    user_encoders=user_encoders,
    item_encoders=item_encoders,
    feature_cfg = feature_config
)

print(dir(fs))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'feature_cfg', 'get_all_item_features', 'get_all_item_ids', 'get_item_features', 'get_user_features', 'item_encoders', 'item_features', 'item_id_tensor', 'item_index', 'map_item_ids_to_indices', 'user_encoders', 'user_features', 'user_index']


## Build Dataset

For this I want a dataset that generates a padded sequence of items for each user. 

First, work out a good max sequence length based on the distribution in the data

In [ ]:
import numpy as np

lens = {k : len(v) for k, v in train_user_item_map.items()}

print("Max len:", max(lens.values()))
print("Avg len:", np.mean(list(lens.values())))
print("Std len:", np.std(list(lens.values())))

Max len: 2314
Avg len: 163.83573439311144
Std len: 190.44394716254214


In [ ]:
max_len = 200

print(f"Pct > {max_len}:", round(len([None for l in lens.values() if l > max_len]) / len(lens) * 100, 4) , "%")

Pct > 200: 25.7824 %


With a max sequence length of 200, we capture the full history for ~75% of users. While this loses some information it allows us to work in a compute limited environment. 

Now let's build the datasets. For sequential recommendation, we need two different dataset types:

1. **Training dataset**: Uses a sliding window approach where, given a sequence of items `[A, B, C, D]`, the model learns to predict each next item from the preceding context: `A→B`, `[A,B]→C`, `[A,B,C]→D`. This is implemented by shifting input and labels by one position.

2. **Evaluation dataset**: Uses the full training history as context and held-out validation items as targets. This mirrors the real inference scenario: given everything we know about a user's past behavior, can we predict what they'll interact with next?

Both datasets use left-padding (padding at the start of sequences) so the most recent item is always at the same position—this works naturally with causal attention where we predict the next token based on previous ones.

In [ ]:
import torch

from torch.utils.data import Dataset, DataLoader

class SequentialTrainDataset(Dataset):
    """Training dataset: generates sequences for next-item prediction."""
    PAD_TOKEN = 0
    
    def __init__(
        self,
        user_item_map: Dict[int, List[int]],
        max_length: int = 50,
        min_length: int = 2,
    ) -> None:
        super().__init__()
        self.max_length = max_length
        self.user_item_map = user_item_map
        self.user_ids = [
            uid for uid, items in user_item_map.items() 
            if len(items) >= min_length
        ]
    
    def __len__(self) -> int:
        return len(self.user_ids)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        user_id = self.user_ids[idx]
        items = self.user_item_map[user_id]
        
        # Need max_length + 1 items to get max_length input/target pairs
        if len(items) > self.max_length + 1:
            items = items[-(self.max_length + 1):]
        
        # input/target shifted by 1
        input_items = items[:-1]
        labels = items[1:]
        actual_len = len(input_items)
        
        # Left-pad to max_length
        pad_len = self.max_length - actual_len
        input_seq = np.full(self.max_length, self.PAD_TOKEN, dtype=np.int64)
        label_seq = np.full(self.max_length, self.PAD_TOKEN, dtype=np.int64)
        input_seq[pad_len:] = input_items
        label_seq[pad_len:] = labels
        
        attention_mask = np.zeros(self.max_length, dtype=np.float32)
        attention_mask[pad_len:] = 1.0
        
        return {
            "user_id": torch.tensor(user_id, dtype=torch.long),
            "input_ids": torch.from_numpy(input_seq),
            "labels": torch.from_numpy(label_seq),
            "attention_mask": torch.from_numpy(attention_mask),
            "seq_length": torch.tensor(actual_len, dtype=torch.long),
        }


class SequentialEvalDataset(Dataset):
    """
    Eval dataset for retrieval metrics.
    
    Returns user's training history as context, and val items as targets.
    Compatible with evaluate_retrieval pattern - model produces scores,
    we compare top-k against val items.
    """
    PAD_TOKEN = 0
    
    def __init__(
        self,
        train_user_item_map: Dict[int, List[int]],
        val_user_item_map: Dict[int, List[int]],
        max_length: int = 50,
    ) -> None:
        super().__init__()
        self.max_length = max_length
        self.train_map = train_user_item_map
        self.val_map = val_user_item_map
        
        # Users with val items AND some training history
        self.user_ids = [
            uid for uid in val_user_item_map
            if len(val_user_item_map[uid]) >= 1 and uid in train_user_item_map
        ]
    
    def __len__(self) -> int:
        return len(self.user_ids)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        user_id = self.user_ids[idx]
        context_items = self.train_map.get(user_id, [])
        target_items = self.val_map[user_id]
        
        # Truncate context to max_length (keep most recent)
        if len(context_items) > self.max_length:
            context_items = context_items[-self.max_length:]

        actual_len = len(context_items)
        pad_len = self.max_length - actual_len
        
        input_seq = np.full(self.max_length, self.PAD_TOKEN, dtype=np.int64)
        input_seq[pad_len:] = context_items
        
        attention_mask = np.zeros(self.max_length, dtype=np.float32)
        attention_mask[pad_len:] = 1.0
        
        return {
            "user_id": torch.tensor(user_id, dtype=torch.long),
            "input_ids": torch.from_numpy(input_seq),
            "attention_mask": torch.from_numpy(attention_mask),
            "seq_length": torch.tensor(actual_len, dtype=torch.long),
            # Targets for metric computation (variable length)
            "target_items": torch.tensor(target_items, dtype=torch.long),
        }

def collate_eval_batches(batch):
    return {
        "user_id" : torch.stack([ex["user_id"] for ex in batch]),
        "input_ids": torch.stack([ex["input_ids"] for ex in batch]),
        "attention_mask": torch.stack([ex["attention_mask"] for ex in batch]),
        "seq_length": torch.stack([ex["seq_length"] for ex in batch]),
        "target_items": [ex["target_items"] for ex in batch],  # keep as list of tensors for ragged
    }

A few implementation details worth noting:

- **PAD_TOKEN = 0**: We reserve index 0 for padding, which aligns with the `+1` offset we built into our encoders earlier
- **Minimum length filtering**: Training requires at least 2 items (one for input, one for target), so we filter out users with singleton interactions
- **Variable-length targets**: The eval dataset keeps targets as a list of tensors rather than padding them, since different users have different numbers of validation interactions. The custom `collate_eval_batches` function handles this ragged structure.

In [ ]:
# Training
train_dataset = SequentialTrainDataset(train_user_item_map, max_length=max_len, min_length=2)

# Evaluation - pass train history as context
val_dataset = SequentialEvalDataset(
    train_user_item_map,    
    val_user_item_map, 
    max_length=max_len
)

In [ ]:
# Check a train sample
train_dataset[0]

{'user_id': tensor(1),
 'input_ids': tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0

In [ ]:
# Check a val sample
val_dataset[0]

{'user_id': tensor(36),
 'input_ids': tensor([2790, 2847, 2850, 2891, 2905, 2919, 2967, 2969, 2971, 2984, 2992, 3004,
         3013, 3019, 3046, 3088, 3107, 3108, 3145, 3185, 3187, 3195, 3230, 3233,
         3240, 3290, 3293, 3356, 3436, 3437, 3457, 3458, 3484, 3510, 3523, 3555,
         3603, 3629, 3644, 3677, 3685, 3717, 3725, 3752, 3767, 3794, 3883,    6,
           10,   16,  109,  110,  160,  163,  164,  227,  233,  346,  374,  451,
          454,  463,  471,  524,  590,  605,  642,  725,  779,  856,  901,  913,
          958, 1063, 1065, 1075, 1077, 1079, 1179, 1180, 1183, 1191, 1193, 1197,
         1210, 1215, 1221, 1246, 1253, 1257, 1280, 1288, 1338, 1350, 1386, 1452,
         1546, 1563, 1569, 1575, 1576, 1600, 1629, 1674, 1743, 1852, 1892, 1960,
         1990, 1996, 2050, 2210, 2220, 2266, 2285, 2323, 2325, 2472, 2696, 2747,
         2803, 2815, 2822, 2848, 2899, 2904, 2998, 3033, 3079, 3128, 3188, 3373,
         3380, 3444, 3459, 3483, 3638, 3659, 3695, 1843, 3340,  258,  29

In [ ]:
batch_size = 16

train_dl = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(val_dataset, batch_size = batch_size, collate_fn = collate_eval_batches)

In [ ]:
train_batch = next(iter(train_dl))
train_batch

{'user_id': tensor([2993, 2311, 5298, 2819, 2634,  144,  230, 4548, 4676,  754, 5792, 3857,
         2707,    3, 2189,   87]),
 'input_ids': tensor([[   0,    0,    0,  ..., 3687, 3741, 3772],
         [   0,    0,    0,  ..., 3257, 3340, 3382],
         [   0,    0,    0,  ..., 3570, 3633, 3634],
         ...,
         [   0,    0,    0,  ..., 3484, 3551, 3603],
         [   0,    0,    0,  ..., 3547, 3686, 3695],
         [   0,    0,    0,  ..., 3604, 3725, 1198]]),
 'labels': tensor([[   0,    0,    0,  ..., 3741, 3772, 3800],
         [   0,    0,    0,  ..., 3340, 3382, 3442],
         [   0,    0,    0,  ..., 3633, 3634, 3635],
         ...,
         [   0,    0,    0,  ..., 3551, 3603, 3799],
         [   0,    0,    0,  ..., 3686, 3695, 3725],
         [   0,    0,    0,  ..., 3725, 1198, 1569]]),
 'attention_mask': tensor([[0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ...,

In [ ]:
val_batch = next(iter(val_dl))
val_batch

{'user_id': tensor([ 36,  59,  65, 102, 131, 146, 157, 164, 169, 184, 192, 193, 195, 229,
         231, 237]),
 'input_ids': tensor([[2790, 2847, 2850,  ..., 3291, 3758, 2706],
         [   0,    0,    0,  ..., 3743, 3747, 3841],
         [   0,    0,    0,  ..., 1193, 2560, 3752],
         ...,
         [   0,    0,    0,  ..., 3687, 3725, 3847],
         [   0,    0,    0,  ..., 2848, 2917, 3459],
         [   0,    0,    0,  ..., 1637,  354, 2041]]),
 'attention_mask': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.]]),
 'seq_length': tensor([200,  93, 119,  32, 200, 200, 200,  25, 200,  28, 200, 177, 200,  83,
          48, 161]),
 'target_items': [tensor([ 245,  294,  592, 1247, 1276, 1654, 2201, 2301, 2626, 2645, 3106, 3571,
          3718]),
  tensor([  17,   25,   32,   58

In [ ]:
val_batch["input_ids"].shape

torch.Size([16, 200])

In [ ]:
train_batch["input_ids"].shape

torch.Size([16, 200])

# Model

## Model Architecture

At its core, GPTRec applies the same autoregressive language modeling approach that powers GPT to sequential recommendation. Just as GPT predicts the next word given previous words, GPTRec predicts the next item given a user's interaction history.

The architecture follows a familiar transformer pattern:

1. **Item Embeddings**: Each item gets a learned embedding vector. We also add positional embeddings so the model knows where each item appears in the sequence.

2. **Causal Transformer**: The key ingredient. Unlike BERT-style models that can look at the full sequence bidirectionally, we use causal (autoregressive) masking—each position can only attend to *earlier* positions. This matches our inference scenario: predict what comes next based only on what we've seen so far.

3. **Weight Tying**: The output projection layer shares weights with the item embedding layer. This is a common trick in language models that reduces parameters and often improves performance—the intuition being that the "meaning" of an item should be consistent whether we're encoding it as input or predicting it as output.

The `compute_loss` method implements standard cross-entropy loss with `ignore_index=0` to skip padding tokens—we only want to learn from real predictions, not from predicting padding.

One notable simplification: unlike the full GPTRec paper which explores SVD-based embedding initialization and various training optimizations, I'm using standard randomly-initialized embeddings here. For a small dataset like this, it should work fine.

In [ ]:
import torch
import torch.nn as nn
import math

class GPTRecModel(nn.Module):
    """
    GPTRec: GPT-style autoregressive transformer for sequential recommendation.
    
    Uses causal masking so each position only attends to previous positions,
    enabling next-item prediction.
    """
    
    def __init__(
        self,
        n_items: int,
        d_model: int = 64,
        n_heads: int = 2,
        n_layers: int = 2,
        d_ff: int = 256,
        max_seq_len: int = 50,
        dropout: float = 0.1,
        pad_token: int = 0,
    ):
        super().__init__()
        self.pad_token = pad_token
        self.d_model = d_model
        
        # Item embedding (+1 for padding token at index 0)
        self.item_embedding = nn.Embedding(n_items + 1, d_model, padding_idx=pad_token)
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)
        
        # Transformer encoder with causal masking
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Output projection to item scores
        self.output_layer = nn.Linear(d_model, n_items + 1)
        self.output_layer.weight = self.item_embedding.weight  # tie weights

    def _generate_causal_mask(self, seq_len: int, device: torch.device) -> torch.Tensor:
        """Generate causal mask: positions can only attend to earlier positions."""
        mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask
    
    def forward(
        self,
        input_ids: torch.Tensor,           # (batch, seq_len)
        attention_mask: torch.Tensor = None,  # (batch, seq_len) - 1 for real, 0 for pad
    ) -> torch.Tensor:
        batch_size, seq_len = input_ids.shape
        device = input_ids.device
        
        # Embeddings
        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.item_embedding(input_ids) + self.pos_embedding(positions)
        x = self.dropout(self.layer_norm(x))
        
        # Causal mask
        causal_mask = self._generate_causal_mask(seq_len, device)
        
        # Padding mask (convert to float: 0.0 = attend, -inf = ignore)
        src_key_padding_mask = torch.where(attention_mask == 1, 0.0, float('-inf'))

        # Transformer
        x = self.transformer(
            x,
            mask=causal_mask,
            src_key_padding_mask=src_key_padding_mask,
        )
        
        # Project to item logits
        logits = self.output_layer(x)  # (batch, seq_len, n_items+1)
        return logits
    
    def compute_loss(self, batch, ignore_index=0):
        """Cross-entropy loss for next-item prediction, ignoring padding."""
        logits = self(batch['input_ids'], batch['attention_mask'])  # (B, seq_len, n_items+1)
        
        # Reshape for cross-entropy: (B*seq_len, n_items+1) vs (B*seq_len,)
        logits_flat = logits.view(-1, logits.size(-1))
        labels_flat = batch['labels'].view(-1)
        
        loss = nn.functional.cross_entropy(
            logits_flat, 
            labels_flat, 
            ignore_index=ignore_index  # Ignore padding positions
        )
        return loss

In [ ]:
n_items = item_cardinalities['item_id']  # 201

model = GPTRecModel(
    n_items=n_items,
    d_model=64,
    n_heads=2,
    n_layers=2,
    d_ff=256,
    max_seq_len=max_len,
    dropout=0.2,
)

# Test forward pass
logits = model(val_batch['input_ids'], val_batch['attention_mask'])
print("Logits shape:", logits.shape)  # (1, seq_len, n_items+1)

Logits shape: torch.Size([16, 200, 3885])


In [ ]:
model.compute_loss(train_batch)

tensor(36.3798, grad_fn=<NllLossBackward0>)

In [ ]:
sum([p.numel() for p in model.parameters()])

365421

## Train / Evaluate

In [ ]:
#from rec.retrieval.metrics import aggregate_retrieval_metrics
from rec.retrieval.metrics import *
from rec.retrieval.metrics import _as_list

# requires rewrite from library version to add dcg for comparison with GPTRec paper. will integrate into library in future iterations
def aggregate_retrieval_metrics(
    topk_indices: torch.Tensor,
    relevant_indices: Sequence[torch.Tensor],
    ks: Iterable[int],
) -> Dict[str, float]:
    ks_list = _as_list(ks)
    if not ks_list or topk_indices.numel() == 0:
        return {}

    max_k = max(ks_list)
    if topk_indices.size(1) < max_k:
        raise ValueError("topk_indices must have at least max(k) columns")

    totals = {f"recall@{k}": 0.0 for k in ks_list}
    totals.update({f"precision@{k}": 0.0 for k in ks_list})
    totals.update({f"dcg@{k}": 0.0 for k in ks_list})
    totals.update({f"ndcg@{k}": 0.0 for k in ks_list})
    totals["mrr"] = 0.0

    num_users = topk_indices.size(0)
    for idx in range(num_users):
        topk = topk_indices[idx]
        rel = relevant_indices[idx]
        if rel.numel() == 0:
            continue
        hits = torch.isin(topk, rel)

        totals["mrr"] += mrr(hits)
        num_rel = int(rel.numel())
        for k in ks_list:
            totals[f"recall@{k}"] += recall_at_k(hits, num_rel, k)
            totals[f"precision@{k}"] += precision_at_k(hits, k)
            dcg = dcg_at_k(hits, k)
            ideal_dcg = idcg_at_k(num_rel, k)
            ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0.0
            totals[f"dcg@{k}"] += dcg
            totals[f"ndcg@{k}"] += ndcg

    if num_users == 0:
        return {}
    return {k: v / float(num_users) for k, v in totals.items()}

def evaluate_gptrec(
    model: GPTRecModel,
    val_dataloader: DataLoader,
    train_user_item_map: Dict[int, List[int]],
    n_items: int,
    ks: List[int] = [5, 10, 20],
    device: torch.device = None,
) -> Dict[str, float]:
    """
    Evaluate GPTRec model on retrieval metrics.
    
    For each user, we:
    1. Get model's logits from the last position (next-item prediction)
    2. Mask out items the user has already seen in training
    3. Get top-k predictions
    4. Compare against validation targets
    """
    if device is None:
        device = next(model.parameters()).device
    
    model.eval()
    
    topk_indices_list = []
    relevant_indices_list = []
    max_k = max(ks)
    
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            user_ids = batch['user_id']
            target_items = batch['target_items']  # list of tensors
            
            # Forward pass
            logits = model(input_ids, attention_mask)  # (B, seq_len, n_items+1)
            
            # Get logits from last position for next-item prediction
            last_logits = logits[:, -1, :]  # (B, n_items+1)
            
            # For each user in batch
            for i in range(len(user_ids)):
                uid = user_ids[i].item()
                scores = last_logits[i].clone()  # (n_items+1,)
                
                # Mask out seen items (set to -inf)
                seen_items = train_user_item_map.get(uid, [])
                if seen_items:
                    seen_tensor = torch.tensor(seen_items, device=device)
                    scores[seen_tensor] = float('-inf')
                
                # Also mask out padding token (index 0)
                scores[0] = float('-inf')
                
                # Get top-k predictions
                topk = torch.topk(scores, min(max_k, n_items)).indices
                topk_indices_list.append(topk.cpu())
                
                # Target items for this user
                relevant_indices_list.append(target_items[i])
    
    if not topk_indices_list:
        return {}
    
    topk_tensor = torch.stack(topk_indices_list, dim=0)
    metrics = aggregate_retrieval_metrics(topk_tensor, relevant_indices_list, ks)
    return metrics

In [ ]:
from tqdm import tqdm

num_epochs = 5
batch_size = 32
lr = 5e-4

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    
    # Training loop
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in pbar:
        optimizer.zero_grad()
        loss = model.compute_loss(batch)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        pbar.set_postfix(loss=f"{loss.item():.2f}")
    
    avg_train_loss = sum(train_losses) / len(train_losses)
    
    # Retrieval metrics (with progress bar)
    metrics = evaluate_gptrec(
        model, val_dl, train_user_item_map,
        n_items=n_items, ks=[5, 10, 20],
    )
    
    # Summary line
    print(f"\nEpoch {epoch+1}: train_loss={avg_train_loss:.4f} | R@10={metrics.get('recall@10', 0):.4f}",  
        f"| DCG@10={metrics.get('dcg@10', 0):.4f} | NDCG@10={metrics.get('ndcg@10', 0):.4f}")

Epoch 1/5:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/189 [00:01<?, ?it/s, loss=36.64]

Epoch 1/5:   1%|          | 1/189 [00:01<05:34,  1.78s/it, loss=36.64]

Epoch 1/5:   1%|          | 1/189 [00:03<05:34,  1.78s/it, loss=36.19]

Epoch 1/5:   1%|          | 2/189 [00:03<05:31,  1.77s/it, loss=36.19]

...

 91%|█████████▏| 21/23 [00:06<00:00,  3.09it/s]

 96%|█████████▌| 22/23 [00:07<00:00,  3.16it/s]

100%|██████████| 23/23 [00:07<00:00,  3.06it/s]

100%|██████████| 23/23 [00:07<00:00,  3.04it/s]


Epoch 5: train_loss=7.7534 | R@10=0.0242 | DCG@10=0.2592 | NDCG@10=0.0612


In [ ]:
metrics

{'recall@5': 0.01575394982720289,
 'recall@10': 0.0241689173872425,
 'recall@20': 0.04311617966492117,
 'precision@5': 0.05770308123249291,
 'precision@10': 0.05546218487394955,
 'precision@20': 0.05042016806722694,
 'dcg@5': 0.17425028015585506,
 'dcg@10': 0.2591816378073866,
 'dcg@20': 0.37284562258827253,
 'ndcg@5': 0.06126104414152299,
 'ndcg@10': 0.061165418642456525,
 'ndcg@20': 0.06258607128889172,
 'mrr': 0.13528584705055305}

The model shows promising results—metrics are improving across epochs and beginning to converge. I'd like to scale up with larger embedding dimensions and more training epochs, but I'm limited by compute in this environment.

One challenge I've encountered is the difficulty of fairly evaluating recommendation models. There's significant variation in how metrics are calculated across papers and implementations, making it hard to compare results directly. I plan to dig deeper into this topic in a future post.

## Cleanup

Remove the rec repo so that the notebook runs end to end on restart, and unneccesary files are removed from the blog.

In [ ]:
!rm -rf rec

In [ ]:
!rm -rf ../../assets/movielens_rec_data

# Conclusion

This post presents a GPTRec implementation using my `rec` framework. The key contributions are:

- A minimal, reproducible PyTorch implementation of GPTRec, contrasting with the [original version](https://github.com/asash/gptrec_rl) which is more general but implemented in TensorFlow
- A successful demonstration of the `rec` framework's capabilities

The model shows reasonable performance, validating that the architecture is implemented correctly.

Looking ahead, I'd like to integrate sequential models into the `rec` framework. The framework currently supports a Retrieval → Ranking pipeline via the [train_all script](https://github.com/AndrewBoney/rec/blob/main/rec/train_all.py). I'm considering two approaches:

1. **Three-stage pipeline** (Retrieval → Sequential → Ranking): The ranking model would either become a hybrid combining traditional ranking with sequential signals, or incorporate the sequential model's logits into the ranking embeddings.

2. **Sequential ranking**: Replace the ranking stage with a sequential model that also leverages user/item features. This aligns with industry trends—see Meta's recent work on [sequence learning for personalized recommendations](https://engineering.fb.com/2024/11/19/data-infrastructure/sequence-learning-personalized-ads-recommendations/).

Finally, a note on tooling: I found solveit's compute limitations frustrating for this implementation-heavy post, requiring many restarts. For future implementation work, I'll likely develop locally and reserve solveit for paper reviews and lighter research tasks.